# Rock Paper Scissors Game Predictor with 1D Convolutional Neural Networks

##### A 1D-CNN applies convolutional operations along one dimension (e.g., time or sequence data). It's ideal for finding patterns in sequential data, like a series of moves in Rock-Paper-Scissors.

In [1]:
# import dependencies
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical
from sklearn.model_selection import train_test_split

## Create data with random sequences for the model to learn

#### Rock = 0, Paper = 1, Scissors = 2

In [2]:
SAMPLE_SIZE = 1000

def generate_data(num_samples=SAMPLE_SIZE, seq_length=5):
    # Initialize empty lists to store input sequences (X) and next moves (y)
    X, y = [], [] 
    
    for _ in range(num_samples):
        sequence = np.random.choice([0, 1, 2], size=seq_length)  # Generate a random sequence of moves
        next_move = (sequence[-1] + 1) % 3  # Calculate the next move based on the rule: "Choose the move that beats the last one"
        
        X.append(sequence) # Append the sequence to X (input data)
        y.append(next_move) # Append the next move to y (output data)
        
    return np.array(X), np.array(y) # Convert X and y to NumPy arrays

In [3]:
X, y = generate_data()
y = to_categorical(y, num_classes=3)  # One-hot encode the output
X = X.reshape((X.shape[0], X.shape[1], 1))  # Reshape for 1D-CNN input

# Split into training and testing sets
# Train size = 70% and Test size = 30% 
# Random state can be any constant number; let it be 50 this time
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=50)

## Build the model

In [ ]:
model = Sequential([
    # first convolutional layer
    Conv1D(64, kernel_size=2, activation='relu', input_shape=(X.shape[1], 1)),
    Dropout(0.2),
    
    # second convolutional layer
    Conv1D(128, kernel_size=2, activation='relu'),
    Dropout(0.2),
    
    # flatten layer
    Flatten(),
    
    # dense layer
    Dense(64, activation='relu'),
    Dense(3, activation='softmax')  # 3 classes: Rock, Paper, Scissors
])

### Model V2: Recurrent Neural Network (RNN)

###### Ignore the code below for 1D CNN

In [ ]:
model = Sequential([
    SimpleRNN(64, activation='relu', input_shape=(X.shape[1], 1)),  # RNN layer
    Dense(32, activation='relu'),  # Fully connected layer
    Dense(3, activation='softmax')  # Output layer (3 classes: Rock, Paper, Scissors)
])

## Training the model

In [5]:
# Compile the model
# Loss function for multi-class classification tasks
# Tracks the accuracy of predictions during training
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy']) 

model.fit(X_train, y_train, epochs=20, batch_size=32, validation_data=(X_test, y_test))
# validation_data=(X_test, y_test): Evaluates performance on the test set after each epoch to track progress.

Epoch 1/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 3s 40ms/step - accuracy: 0.3990 - loss: 1.0779 - val_accuracy: 0.6933 - val_loss: 0.9297
Epoch 2/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step - accuracy: 0.7105 - loss: 0.8581 - val_accuracy: 0.9200 - val_loss: 0.5050
Epoch 3/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step - accuracy: 0.8979 - loss: 0.4378 - val_accuracy: 0.9367 - val_loss: 0.1896
Epoch 4/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9355 - loss: 0.1981 - val_accuracy: 0.9600 - val_loss: 0.0936
Epoch 5/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9491 - loss: 0.1206 - val_accuracy: 0.9833 - val_loss: 0.0644
Epoch 6/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 11ms/step - accuracy: 0.9734 - loss: 0.0806 - val_accuracy: 1.0000 - val_loss: 0.0261
Epoch 7/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step - accuracy: 0.9902 - loss: 0.0566 - val_accuracy: 1.0000 - val_loss: 0.0158
Epoch 8/20
22/22 ━━━━━━━━━━━━━━━━━━━━ 0s 12ms/step - accuracy: 0.9938 - loss: 0.0377 - val_accuracy: 1.0000 - v

## Testing the model

In [6]:
# generates a random sequence of rock, paper, scissors for testing the model
def generate_test_data(sequence_length):
    # Generate a random sequence of moves (0, 1, or 2)
    sequence = np.random.choice([0, 1, 2], size=sequence_length)
    
    # Convert numbers to Rock, Paper, Scissors
    moves = []
    
    for move in sequence:
        if move == 0:
            moves.append("Rock")
        elif move == 1:
            moves.append("Paper")
        else:
            moves.append("Scissors")
            
    # Format the sequence for the model
    sequence = sequence.reshape(1, sequence_length, 1)
    
    # Print the sequence in a readable format
    print(f"The new test sequence is: {moves}")
    
    return sequence

In [7]:
test_sequence = generate_test_data(sequence_length=5)

# Ensures the test sequence is correctly reshaped for the model's input format 
test_sequence = test_sequence.reshape((1, test_sequence.shape[1], 1))

# Feeds the test sequence into the model to predict the next move. 
predicted = model.predict(test_sequence)

The new test sequence is: ['Rock', 'Paper', 'Scissors', 'Scissors', 'Paper']
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 170ms/step


### The predicted is an output of the model which is possibilities of each class.
### For example: it could be [0.1, 0.7, 02] for Rock = 10%, Paper = 70%, and Scissors = 20%. 
### np.argmax will choose the max value out of them, which is 0.7 (Paper) in this case.

In [8]:
predicted_move = np.argmax(predicted)

print(f"Predicted Move: {['Rock', 'Paper', 'Scissors'][predicted_move]}")

Predicted Move: Scissors
